### ¿Qué hace este script?
Para cada sitio y fecha calcula:
* usuarios.nivelesSatisfaccionUsuarios: Clasificación de la respuesta (bueno, regular, malo)
* usuarios.nivelesSatisfaccionUsuariosRespuesta: Total de respuestas para cada opción
* usuarios.nivelesSatisfaccionUsuariosRespuestaPorCien (porcentaje asociado al total para ese grupo de respuetas)
* usuarios.cantidadCalificaciones: total de calificaciones (incluye todas las opciones)

In [56]:
from elasticsearch import Elasticsearch, helpers
import pandas as pd
import numpy as np
from ssl import create_default_context
from datetime import datetime, timedelta
import time
import parametros #parametros propios del desarrollo

### Conexión a Elastic Search

In [57]:
context = create_default_context(cafile=parametros.cafile)
es = Elasticsearch(
    parametros.servidor,
    http_auth=(parametros.usuario_EC, parametros.password_EC),
    scheme="https",
    port=parametros.puerto,
    ssl_context=context,
    timeout=60, max_retries=3, retry_on_timeout=True
)

### Calculando fechas para la ejecución

In [58]:
now = datetime.now()
fecha_hoy = str(now.strftime("%Y.%m.%d"))

### nombre de indice donde se insertará

se define tanto el indice principal como el que controla la ejecución

In [59]:
indice = parametros.usuarios_tablero13_index
indice_control = parametros.tableros_mintic_control

### Funcion que genera JSON compatible con ElasticSearch

In [60]:
def filterKeys(document):
    return {key: document[key] for key in use_these_keys }

### Trae la ultima fecha para control de ejecución

Cuando en el rango de tiempo de la ejecución, no se insertan nuevos valores, las fecha maxima en indice mintic no aumenta, por tanto se usa esta fecha de control para garantizar que incremente el bucle de ejecución

In [61]:
total_docs = 1
try:
    response = es.search(
        index= indice_control,
        body={
               "_source": ["valoraciones_usuario"],
              "query": {
                "bool": {
                  "filter": [
                  {
                    "exists": {
                      "field":"jerarquia-tablero13"
                    }
                  }
                  ]
                }
              }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        fecha_ejecucion = doc["_source"]['valoraciones_usuario']
except:
    fecha_ejecucion = '2021-06-01 00:00:00'
if response["hits"]["hits"] == []:
    fecha_ejecucion = '2021-06-01 00:00:00'
print("ultima fecha para control de ejecucion:",fecha_ejecucion)

ultima fecha para control de ejecucion: 2021-06-01 00:00:00


### leyendo indice semilla-inventario

En el script que ingesta semilla, trae la información de los centros de conexión administrados. Para el indice principal se requiere:
* site_id como llave del centro de conexión.
* Datos geográficos (Departamento, municipio, centro poblado, sede, energía, latitud, longitud,  COD_ISO , id_Beneficiario).

In [62]:
total_docs = 10000
try:
    response = es.search(
        index= parametros.semilla_inventario_index,
        body={
               "_source": ['site_id','nombre_municipio', 'nombre_departamento', 'nombre_centro_pob', 'nombreSede' 
                           , 'energiadesc', 'latitud', 'longitud', 'COD_ISO','id_Beneficiario']
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    datos_semilla = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) #pd.DataFrame(fields)
except:
    print ("Error")

### Cambiando nombre de campos y generando location

* Se valida latitud y longitud. Luego se calcula campo location
* Se renombran los campos de semilla

In [63]:
import re
def get_location(x):
    patron = re.compile('^(\-?\d+(\.\d+)?),\s*(\-?\d+(\.\d+)?)$') #patrón que debe cumplir
    if (not patron.match(x) is None):
        return x.replace(',','.')
    else:
        #Código a ejecutar si las coordenadas no son válidas
        return 'a'
datos_semilla['latitud'] = datos_semilla['latitud'].apply(get_location)
datos_semilla['longitud'] = datos_semilla['longitud'].apply(get_location)
datos_semilla = datos_semilla.drop(datos_semilla[(datos_semilla["longitud"]=='a') | (datos_semilla["latitud"]=='a')].index)
datos_semilla['usuarios.tablero13.location'] = datos_semilla['latitud'] + ',' + datos_semilla['longitud']
datos_semilla['usuarios.tablero13.location']=datos_semilla['usuarios.tablero13.location'].str.replace('a,a','')
datos_semilla.drop(columns=['latitud','longitud'],inplace=True)

In [64]:
datos_semilla = datos_semilla.rename(columns={'lugar_cod' : 'usuarios.tablero13.centroDigitalUsuarios'
                                            , 'nombre_municipio': 'usuarios.tablero13.nombreMunicipio'
                                            , 'nombre_departamento' : 'usuarios.tablero13.nombreDepartamento'
                                            , 'nombre_centro_pob': 'usuarios.tablero13.localidad'
                                            , 'nombreSede' : 'usuarios.tablero13.nomCentroDigital'
                                            , 'energiadesc' : 'usuarios.tablero13.sistemaEnergia'
                                            , 'COD_ISO' : 'usuarios.tablero13.codISO'
                                            , 'id_Beneficiario' : 'usuarios.tablero13.idBeneficiario'})

Se descartan los registros que tengan la latitud y longitud vacía o no valida

In [65]:
datos_semilla = datos_semilla.drop(datos_semilla[(datos_semilla["usuarios.tablero13.location"]=='')].index)

## leyendo indice ohmyfi valoraciones

Se toman en cuenta todas las valoraciones realizadas dentro del rango de fecha fecha. Campos leidos:

* lugar_cod que es la llave para cruzar con site_id.
* datos asociados a la valoración: pregunta, respuesta, fechahora (cuando se registró la valoración)

In [66]:
def trae_valoraciones(fecha_ini,fecha_fin):
    total_docs = 10000
    response = es.search(
        index= parametros.ohmyfi_val_index,
        body={
                  "_source": ["lugar_cod", "respuesta","fechahora","@timestamp"]
                , "query": {
                      "range": {
                            "fechahora": {
                            "gte": fecha_ini,
                            "lt":  fecha_fin
                            }
                        }
                  }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    return pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) 


### Se realiza la consulta de datos

* Se calcula rango en base a la fecha de control. Para este caso es de un día.
* Se ejecuta la función de consulta con el rango de fechas.
* Si no retorna datos se incrementa el rango y se ejecuta nuevamente. Este proceso se repite hasta conseguir datos o hasta que el rango de ejecución alcance la fecha y hora actual.

In [67]:
fecha_max_mintic = fecha_ejecucion
fecha_tope_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(days=1)-timedelta(seconds=1)).strftime("%Y-%m-%d %H:%M:%S")
datos_valoraciones =  trae_valoraciones(fecha_max_mintic,fecha_tope_mintic)

if datos_valoraciones is None or datos_valoraciones.empty:
    while (datos_valoraciones is None or datos_valoraciones.empty) and ((datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S').strftime("%Y-%m-%d %H:%M:%S")) < str(now.strftime("%Y-%m-%d %H:%M:%S"))):
        fecha_max_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")
        fecha_tope_mintic = (datetime.strptime(fecha_tope_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")
        datos_valoraciones = trae_valoraciones(fecha_max_mintic,fecha_tope_mintic)
else:
    pass

1. Se descartan las respuestas Si y No de las valoraciones, las cuales corresponden a la pregunta: "Te gustaría calificar tu última conexión en". De esta forma solo se dejan las respuestas asociadas a la percepción de calidad por parte del usuario.
2. Se estandariza lugar_cod a site_id
3. Se estadariza fecha para agrupar (solo se toma yyyy-mm-dd)
4. Se Calcula nivel de valoracion por pregunta y total de valoraciones

* Para cada centro de conexión, pregunta, se contabilizan las valoraciones. El calculo es diario

In [68]:
try:
    datos_valoraciones = datos_valoraciones.drop(datos_valoraciones[(datos_valoraciones["respuesta"].isin(['Si','No']))].index)
    #datos_valoraciones['mac_usuario'] = datos_valoraciones['mac_usuario'].str.replace('-',':')
    datos_valoraciones = datos_valoraciones.rename(columns={'lugar_cod': 'site_id'})
    datos_valoraciones['fecha'] = datos_valoraciones["fechahora"].str.split(" ", n = 1, expand = True)[0]
    nivel_valoraciones=datos_valoraciones[['fechahora', 'site_id'
                                         , 'respuesta'
                                         , 'fecha']].groupby(['site_id','respuesta','fecha']).agg(['count']).reset_index()
    nivel_valoraciones.columns = nivel_valoraciones.columns.droplevel(1)
    nivel_valoraciones = nivel_valoraciones.rename(columns={'fechahora' :'usuarios.tablero13.nivelesSatisfaccionUsuarios'
                                                           ,'respuesta' :'usuarios.tablero13.nivelesSatisfaccionUsuariosRespuesta'
                                                           ,'fecha' : 'usuarios.tablero13.fechaCalificacion'})
    
    total_valoraciones = datos_valoraciones[['site_id','fechahora','fecha']].groupby(['site_id','fecha']).agg(['count']).reset_index()
    total_valoraciones.columns = total_valoraciones.columns.droplevel(1)
    total_valoraciones = total_valoraciones.rename(columns={'fechahora' :'usuarios.tablero13.cantidadCalificaciones'
                                                           ,'fecha' : 'usuarios.tablero13.fechaCalificacion'})

    #Con ambos se calcula el porcentaje de cada respuesta
    nivel_valoraciones =  pd.merge(nivel_valoraciones,total_valoraciones, on=['site_id','usuarios.tablero13.fechaCalificacion'],how='inner')
    nivel_valoraciones['usuarios.tablero13.nivelesSatisfaccionUsuariosRespuestaPorCien'] = ((nivel_valoraciones['usuarios.tablero13.nivelesSatisfaccionUsuarios']) / nivel_valoraciones['usuarios.tablero13.cantidadCalificaciones']).round(4)
except:
    total_valoraciones = pd.DataFrame(columns=['site_id','usuarios.tablero13.fechaCalificacion'
                                              ,'usuarios.tablero13.cantidadCalificaciones'])
    nivel_valoraciones = pd.DataFrame(columns=['site_id','usuarios.tablero13.fechaCalificacion'
                                              ,'usuarios.tablero13.nivelesSatisfaccionUsuarios'
                                              ,'usuarios.tablero13.nivelesSatisfaccionUsuariosRespuesta'
                                              ,'usuarios.tablero13.nivelesSatisfaccionUsuariosRespuestaPorCien'])
    

In [69]:
total_valoraciones

,site_id,usuarios.tablero13.fechaCalificacion,usuarios.tablero13.cantidadCalificaciones
0,10835-ZGYO372,2021-06-01,3
1,11107-ZGYO062,2021-06-01,20
2,11113-ZGYO059,2021-06-01,10
3,13253-ZGYO527,2021-06-01,11
4,13304-ZGYO085,2021-06-01,16
...,...,...,...
449,74271-ZGYO207,2021-06-01,2
450,74272-ZZZY510,2021-06-01,8
451,74281-ZGYO779,2021-06-01,16
452,74366-ZGYO061,2021-06-01,9


In [70]:
nivel_valoraciones

,site_id,usuarios.tablero13.nivelesSatisfaccionUsuariosRespuesta,usuarios.tablero13.fechaCalificacion,usuarios.tablero13.nivelesSatisfaccionUsuarios,usuarios.tablero13.cantidadCalificaciones,usuarios.tablero13.nivelesSatisfaccionUsuariosRespuestaPorCien
0,10835-ZGYO372,Excelente,2021-06-01,3,3,1.0000
1,11107-ZGYO062,Aceptable,2021-06-01,6,20,0.3000
2,11107-ZGYO062,Buena,2021-06-01,5,20,0.2500
3,11107-ZGYO062,Excelente,2021-06-01,9,20,0.4500
4,11113-ZGYO059,Aceptable,2021-06-01,2,10,0.2000
...,...,...,...,...,...,...
1108,74366-ZGYO061,Buena,2021-06-01,3,9,0.3333
1109,74366-ZGYO061,Excelente,2021-06-01,4,9,0.4444
1110,74367-ZGYO181,Aceptable,2021-06-01,1,14,0.0714
1111,74367-ZGYO181,Buena,2021-06-01,2,14,0.1429


### Cruzando con semilla las agregaciones de valoraciones

In [71]:
mintic_valoraciones = pd.merge(datos_semilla, nivel_valoraciones, on='site_id',how='inner')

In [72]:
mintic_valoraciones

,usuarios.tablero13.nomCentroDigital,usuarios.tablero13.codISO,usuarios.tablero13.localidad,site_id,usuarios.tablero13.nombreDepartamento,usuarios.tablero13.sistemaEnergia,usuarios.tablero13.nombreMunicipio,usuarios.tablero13.idBeneficiario,usuarios.tablero13.location,usuarios.tablero13.nivelesSatisfaccionUsuariosRespuesta,usuarios.tablero13.fechaCalificacion,usuarios.tablero13.nivelesSatisfaccionUsuarios,usuarios.tablero13.cantidadCalificaciones,usuarios.tablero13.nivelesSatisfaccionUsuariosRespuestaPorCien
0,C.E.R LA PAULINA,CO-ANT,LA PAULINA,22154-ZGYO176,ANTIOQUIA,RED INTERCONECTADA,VALDIVIA,22154,"6.7048548,-76.14525469",Buena,2021-06-01,2,11,0.1818
1,C.E.R LA PAULINA,CO-ANT,LA PAULINA,22154-ZGYO176,ANTIOQUIA,RED INTERCONECTADA,VALDIVIA,22154,"6.7048548,-76.14525469",Excelente,2021-06-01,8,11,0.7273
2,C.E.R LA PAULINA,CO-ANT,LA PAULINA,22154-ZGYO176,ANTIOQUIA,RED INTERCONECTADA,VALDIVIA,22154,"6.7048548,-76.14525469",Regular,2021-06-01,1,11,0.0909
3,CENTRO EDUCATIVO RURAL EL CERRO,CO-ANT,EL CERRO,20022-ZGYO177,ANTIOQUIA,RED INTERCONECTADA,FRONTINO,20022,"4.225166667,-73.10716667",Buena,2021-06-01,1,6,0.1667
4,CENTRO EDUCATIVO RURAL EL CERRO,CO-ANT,EL CERRO,20022-ZGYO177,ANTIOQUIA,RED INTERCONECTADA,FRONTINO,20022,"4.225166667,-73.10716667",Excelente,2021-06-01,1,6,0.1667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,I.E. PABLO VI - SEDE PRINCIPAL,CO-TOL,INSP POL PLAYA RICA,49402-ZGYO895,TOLIMA,RED INTERCONECTADA,SAN ANTONIO,49402,"4.01721444,-75.464945510",Aceptable,2021-06-01,1,6,0.1667
986,I.E. PABLO VI - SEDE PRINCIPAL,CO-TOL,INSP POL PLAYA RICA,49402-ZGYO895,TOLIMA,RED INTERCONECTADA,SAN ANTONIO,49402,"4.01721444,-75.464945510",Buena,2021-06-01,2,6,0.3333
987,I.E. PABLO VI - SEDE PRINCIPAL,CO-TOL,INSP POL PLAYA RICA,49402-ZGYO895,TOLIMA,RED INTERCONECTADA,SAN ANTONIO,49402,"4.01721444,-75.464945510",Excelente,2021-06-01,3,6,0.5000
988,MESETAS,CO-TOL,VDA MESETAS,49410-ZZZY519,TOLIMA,RED INTERCONECTADA,SAN ANTONIO,49410,"3.89944726,-75.52913659",Buena,2021-06-01,5,7,0.7143


# Escribiendo en indice la información de Valoraciones

Se convierten los nulos a ceros a nivelesSatisfaccionUsuarios y cantidadCalificaciones

In [73]:
try:
    mintic_valoraciones.fillna({'usuarios.tablero13.nivelesSatisfaccionUsuarios':0
                               ,'usuarios.tablero13.nivelesSatisfaccionUsuariosRespuestaPorCien':0},inplace=True)
    mintic_valoraciones[['usuarios.tablero13.nivelesSatisfaccionUsuarios']] = mintic_valoraciones[['usuarios.tablero13.nivelesSatisfaccionUsuarios']].astype(int)
    mintic_valoraciones = mintic_valoraciones.rename(columns={'site_id' : 'usuarios.tablero13.siteID'})
    mintic_valoraciones.dropna(subset=['usuarios.tablero13.nivelesSatisfaccionUsuariosRespuesta'], inplace=True)
    mintic_valoraciones["usuarios.tablero13.anyo"] = mintic_valoraciones["usuarios.tablero13.fechaCalificacion"].str[0:4]
    mintic_valoraciones["usuarios.tablero13.mes"] = mintic_valoraciones["usuarios.tablero13.fechaCalificacion"].str[5:7]
    mintic_valoraciones["usuarios.tablero13.dia"] = mintic_valoraciones["usuarios.tablero13.fechaCalificacion"].str[8:10]
    mintic_valoraciones['nombreDepartamento'] = mintic_valoraciones['usuarios.tablero13.nombreDepartamento']
    mintic_valoraciones['nombreMunicipio'] = mintic_valoraciones['usuarios.tablero13.nombreMunicipio']
    mintic_valoraciones['idBeneficiario'] = mintic_valoraciones['usuarios.tablero13.idBeneficiario']
    mintic_valoraciones['fecha'] = mintic_valoraciones['usuarios.tablero13.fechaCalificacion']
    mintic_valoraciones['anyo'] = mintic_valoraciones['usuarios.tablero13.anyo']
    mintic_valoraciones['mes'] = mintic_valoraciones['usuarios.tablero13.mes']
    mintic_valoraciones['dia'] = mintic_valoraciones['usuarios.tablero13.dia']
except:
    
    print('Null')
    
    pass

In [74]:
mintic_valoraciones.dtypes

usuarios.tablero13.nomCentroDigital                                object
usuarios.tablero13.codISO                                          object
usuarios.tablero13.localidad                                       object
usuarios.tablero13.siteID                                          object
usuarios.tablero13.nombreDepartamento                              object
usuarios.tablero13.sistemaEnergia                                  object
usuarios.tablero13.nombreMunicipio                                 object
usuarios.tablero13.idBeneficiario                                   int64
usuarios.tablero13.location                                        object
usuarios.tablero13.nivelesSatisfaccionUsuariosRespuesta            object
usuarios.tablero13.fechaCalificacion                               object
usuarios.tablero13.nivelesSatisfaccionUsuarios                      int64
usuarios.tablero13.cantidadCalificaciones                           int64
usuarios.tablero13.nivelesSatisfaccion

In [75]:
mintic_valoraciones

,usuarios.tablero13.nomCentroDigital,usuarios.tablero13.codISO,usuarios.tablero13.localidad,usuarios.tablero13.siteID,usuarios.tablero13.nombreDepartamento,usuarios.tablero13.sistemaEnergia,usuarios.tablero13.nombreMunicipio,usuarios.tablero13.idBeneficiario,usuarios.tablero13.location,usuarios.tablero13.nivelesSatisfaccionUsuariosRespuesta,...,usuarios.tablero13.anyo,usuarios.tablero13.mes,usuarios.tablero13.dia,nombreDepartamento,nombreMunicipio,idBeneficiario,fecha,anyo,mes,dia
0,C.E.R LA PAULINA,CO-ANT,LA PAULINA,22154-ZGYO176,ANTIOQUIA,RED INTERCONECTADA,VALDIVIA,22154,"6.7048548,-76.14525469",Buena,...,2021,06,01,ANTIOQUIA,VALDIVIA,22154,2021-06-01,2021,06,01
1,C.E.R LA PAULINA,CO-ANT,LA PAULINA,22154-ZGYO176,ANTIOQUIA,RED INTERCONECTADA,VALDIVIA,22154,"6.7048548,-76.14525469",Excelente,...,2021,06,01,ANTIOQUIA,VALDIVIA,22154,2021-06-01,2021,06,01
2,C.E.R LA PAULINA,CO-ANT,LA PAULINA,22154-ZGYO176,ANTIOQUIA,RED INTERCONECTADA,VALDIVIA,22154,"6.7048548,-76.14525469",Regular,...,2021,06,01,ANTIOQUIA,VALDIVIA,22154,2021-06-01,2021,06,01
3,CENTRO EDUCATIVO RURAL EL CERRO,CO-ANT,EL CERRO,20022-ZGYO177,ANTIOQUIA,RED INTERCONECTADA,FRONTINO,20022,"4.225166667,-73.10716667",Buena,...,2021,06,01,ANTIOQUIA,FRONTINO,20022,2021-06-01,2021,06,01
4,CENTRO EDUCATIVO RURAL EL CERRO,CO-ANT,EL CERRO,20022-ZGYO177,ANTIOQUIA,RED INTERCONECTADA,FRONTINO,20022,"4.225166667,-73.10716667",Excelente,...,2021,06,01,ANTIOQUIA,FRONTINO,20022,2021-06-01,2021,06,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,I.E. PABLO VI - SEDE PRINCIPAL,CO-TOL,INSP POL PLAYA RICA,49402-ZGYO895,TOLIMA,RED INTERCONECTADA,SAN ANTONIO,49402,"4.01721444,-75.464945510",Aceptable,...,2021,06,01,TOLIMA,SAN ANTONIO,49402,2021-06-01,2021,06,01
986,I.E. PABLO VI - SEDE PRINCIPAL,CO-TOL,INSP POL PLAYA RICA,49402-ZGYO895,TOLIMA,RED INTERCONECTADA,SAN ANTONIO,49402,"4.01721444,-75.464945510",Buena,...,2021,06,01,TOLIMA,SAN ANTONIO,49402,2021-06-01,2021,06,01
987,I.E. PABLO VI - SEDE PRINCIPAL,CO-TOL,INSP POL PLAYA RICA,49402-ZGYO895,TOLIMA,RED INTERCONECTADA,SAN ANTONIO,49402,"4.01721444,-75.464945510",Excelente,...,2021,06,01,TOLIMA,SAN ANTONIO,49402,2021-06-01,2021,06,01
988,MESETAS,CO-TOL,VDA MESETAS,49410-ZZZY519,TOLIMA,RED INTERCONECTADA,SAN ANTONIO,49410,"3.89944726,-75.52913659",Buena,...,2021,06,01,TOLIMA,SAN ANTONIO,49410,2021-06-01,2021,06,01


In [76]:
mintic_valoraciones.dtypes

usuarios.tablero13.nomCentroDigital                                object
usuarios.tablero13.codISO                                          object
usuarios.tablero13.localidad                                       object
usuarios.tablero13.siteID                                          object
usuarios.tablero13.nombreDepartamento                              object
usuarios.tablero13.sistemaEnergia                                  object
usuarios.tablero13.nombreMunicipio                                 object
usuarios.tablero13.idBeneficiario                                   int64
usuarios.tablero13.location                                        object
usuarios.tablero13.nivelesSatisfaccionUsuariosRespuesta            object
usuarios.tablero13.fechaCalificacion                               object
usuarios.tablero13.nivelesSatisfaccionUsuarios                      int64
usuarios.tablero13.cantidadCalificaciones                           int64
usuarios.tablero13.nivelesSatisfaccion

In [77]:
use_these_keys = ['usuarios.tablero13.nomCentroDigital'
                  , 'usuarios.tablero13.codISO'
                  , 'usuarios.tablero13.localidad'
                  , 'usuarios.tablero13.siteID'
                  , 'usuarios.tablero13.nombreDepartamento'
                  , 'usuarios.tablero13.sistemaEnergia'
                  , 'usuarios.tablero13.nombreMunicipio'
                  , 'usuarios.tablero13.idBeneficiario'
                  , 'usuarios.tablero13.location'
                  , 'usuarios.tablero13.nivelesSatisfaccionUsuariosRespuesta'
                  , 'usuarios.tablero13.fechaCalificacion'
                  , 'usuarios.tablero13.nivelesSatisfaccionUsuarios'
                  , 'usuarios.tablero13.nivelesSatisfaccionUsuariosRespuestaPorCien'               
                  , 'usuarios.tablero13.anyo'
                  , 'usuarios.tablero13.mes'
                  , 'usuarios.tablero13.dia'
                    , 'nombreDepartamento'
                    , 'nombreMunicipio'
                    , 'idBeneficiario'
                    , 'fecha'
                    , 'anyo'
                    , 'mes'
                    , 'dia'
                  , '@timestamp']
try:
    mintic_valoraciones['@timestamp'] = now.isoformat()
    def doc_generator(df):
        df_iter = df.iterrows()
        for index, document in df_iter:
            yield {
                    "_index": indice, 
                    "_id": f"{'Valoracion-' + str(document['usuarios.tablero13.siteID']) + '-' + str(document['usuarios.tablero13.fechaCalificacion']) + '-' + str(document['usuarios.tablero13.nivelesSatisfaccionUsuariosRespuesta'])}",
                    "_source": filterKeys(document),
                }
    salida = helpers.bulk(es, doc_generator(mintic_valoraciones))
    print("Fecha: ", now,"- Valoraciones insertadas en indice principal:",salida[0])
except:
    print("Fecha: ", now,"- No se insertaron valoraciones en indice principal")

Fecha:  2021-07-28 14:42:19.122926 - Valoraciones insertadas en indice principal: 990


## Insertando total de calificaciones

In [78]:
mintic_calificaciones = pd.merge(datos_semilla,  total_valoraciones, on='site_id', how='inner')

In [79]:
mintic_calificaciones

,usuarios.tablero13.nomCentroDigital,usuarios.tablero13.codISO,usuarios.tablero13.localidad,site_id,usuarios.tablero13.nombreDepartamento,usuarios.tablero13.sistemaEnergia,usuarios.tablero13.nombreMunicipio,usuarios.tablero13.idBeneficiario,usuarios.tablero13.location,usuarios.tablero13.fechaCalificacion,usuarios.tablero13.cantidadCalificaciones
0,C.E.R LA PAULINA,CO-ANT,LA PAULINA,22154-ZGYO176,ANTIOQUIA,RED INTERCONECTADA,VALDIVIA,22154,"6.7048548,-76.14525469",2021-06-01,11
1,CENTRO EDUCATIVO RURAL EL CERRO,CO-ANT,EL CERRO,20022-ZGYO177,ANTIOQUIA,RED INTERCONECTADA,FRONTINO,20022,"4.225166667,-73.10716667",2021-06-01,6
2,BUENOS AIRES,CO-ANT,VEREDA BUENOS AIRES,74367-ZGYO181,ANTIOQUIA,RED INTERCONECTADA,VALDIVIA,74367,"5.25634991,-75.74272893",2021-06-01,14
3,C.E.R LOS NUTABES,CO-ANT,EL QUINCE,22139-ZGYO180,ANTIOQUIA,RED INTERCONECTADA,VALDIVIA,22139,"8.12043461,-75.56265199",2021-06-01,21
4,I. E. R. MULATICOS PIEDRECITAS,CO-ANT,VDA. MULATICOS,20696-ZGYO163,ANTIOQUIA,RED INTERCONECTADA,NECOCLÍ,20696,"8.07331926,-75.65995723",2021-06-01,7
...,...,...,...,...,...,...,...,...,...,...,...
398,CALARMA,CO-TOL,VDA CALARMA,49381-ZZZY529,TOLIMA,RED INTERCONECTADA,SAN ANTONIO,49381,"3.88306069,-75.45596105",2021-06-01,3
399,LA MESETA VENTILLAS,CO-TOL,MESETAS VENTANILLA,49394-ZZZY518,TOLIMA,RED INTERCONECTADA,SAN ANTONIO,49394,"3.891111111,-75.54055556",2021-06-01,7
400,SANTA ROSA DE TETUAN,CO-TOL,VDA SANTA ROSA,49396-ZZZY516,TOLIMA,RED INTERCONECTADA,SAN ANTONIO,49396,"4.10383431,-75.423253240",2021-06-01,9
401,I.E. PABLO VI - SEDE PRINCIPAL,CO-TOL,INSP POL PLAYA RICA,49402-ZGYO895,TOLIMA,RED INTERCONECTADA,SAN ANTONIO,49402,"4.01721444,-75.464945510",2021-06-01,6


In [80]:
try:
    mintic_calificaciones.fillna({'usuarios.tablero13.cantidadCalificaciones':0},inplace=True)
    mintic_calificaciones[['usuarios.tablero13.cantidadCalificaciones']] = mintic_calificaciones[['usuarios.tablero13.cantidadCalificaciones']].astype(int)
    mintic_calificaciones = mintic_calificaciones.rename(columns={'site_id' : 'usuarios.siteID'})
    mintic_calificaciones.dropna(subset=['usuarios.tablero13.cantidadCalificaciones'], inplace=True)
    mintic_calificaciones["usuarios.tablero13.anyo"] = mintic_calificaciones["usuarios.tablero13.fechaCalificacion"].str[0:4]
    mintic_calificaciones["usuarios.tablero13.mes"] = mintic_calificaciones["usuarios.tablero13.fechaCalificacion"].str[5:7]
    mintic_calificaciones["usuarios.tablero13.dia"] = mintic_calificaciones["usuarios.tablero13.fechaCalificacion"].str[8:10]
    mintic_calificaciones['nombreDepartamento'] = mintic_calificaciones['usuarios.tablero13.nombreDepartamento']
    mintic_calificaciones['nombreMunicipio'] = mintic_calificaciones['usuarios.tablero13.nombreMunicipio']
    mintic_calificaciones['idBeneficiario'] = mintic_calificaciones['usuarios.idBeneficiario']
    mintic_calificaciones['fecha'] = mintic_calificaciones['usuarios.tablero13.fechaCalificacion']
    mintic_calificaciones['anyo'] = mintic_calificaciones['usuarios.tablero13.anyo']
    mintic_calificaciones['mes'] = mintic_calificaciones['usuarios.tablero13.mes']
    mintic_calificaciones['dia'] = mintic_calificaciones['usuarios.tablero13.dia']
except:

    pass

In [81]:
mintic_calificaciones.dtypes

usuarios.tablero13.nomCentroDigital          object
usuarios.tablero13.codISO                    object
usuarios.tablero13.localidad                 object
usuarios.siteID                              object
usuarios.tablero13.nombreDepartamento        object
usuarios.tablero13.sistemaEnergia            object
usuarios.tablero13.nombreMunicipio           object
usuarios.tablero13.idBeneficiario             int64
usuarios.tablero13.location                  object
usuarios.tablero13.fechaCalificacion         object
usuarios.tablero13.cantidadCalificaciones     int64
usuarios.tablero13.anyo                      object
usuarios.tablero13.mes                       object
usuarios.tablero13.dia                       object
nombreDepartamento                           object
nombreMunicipio                              object
dtype: object

In [86]:
use_these_keys = ['usuarios.tablero13.nomCentroDigital'
                  , 'usuarios.tablero13.codISO'
                  , 'idBeneficiario'
                  , 'usuarios.tablero13.localidad'
                  , 'usuarios.tablero13.siteID'
                  , 'usuarios.tablero13.nombreDepartamento'
                  , 'usuarios.tablero13.sistemaEnergia'
                  , 'usuarios.tablero13.nombreMunicipio'
                  , 'usuarios.tablero13.location'
                  , 'usuarios.tablero13.cantidadCalificaciones'
                  , 'usuarios.tablero13.fechaCalificacion'
                  , 'usuarios.tablero13.anyo'
                  , 'usuarios.tablero13.mes'
                  , 'usuarios.tablero13.dia'
                    , 'nombreDepartamento'
                    , 'nombreMunicipio'
                  , '@timestamp']
try:
    mintic_calificaciones['@timestamp'] = now.isoformat()
    def doc_generator(df):
        df_iter = df.iterrows()
        for index, document in df_iter:
            yield {
                    "_index": indice, 
                    "_id": f"{ 'Calificacion-' + str(document['usuarios.tablero13.siteID']) + '-' + str(document['usuarios.tablero13.fechaCalificacion'])}",
                    "_source": filterKeys(document),
                }
    salida = helpers.bulk(es, doc_generator(mintic_calificaciones))
    print("Fecha: ", now,"- Total calificaciones insertadas en indice principal:",salida[0])
except:
    print("Fecha: ", now,"- No se insertaron totales de calificaciones en indice principal")

Fecha:  2021-07-28 14:42:19.122926 - No se insertaron totales de calificaciones en indice principal


In [87]:
"usuarios.tablero13.siteID" in mintic_calificaciones.columns

False

### Guardando fecha para control de ejecución

* Se actualiza la fecha de control. Si el calculo supera la fecha hora actual, se asocia esta ultima.

In [88]:
fecha_ejecucion = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")
if fecha_ejecucion > str(now.strftime('%Y-%m-%d %H:%M:%S'))[0:10] + ' 00:00:00':
    fecha_ejecucion = str(now.strftime('%Y-%m-%d %H:%M:%S'))[0:10] + ' 00:00:00'
response = es.index(
        index = indice_control,
        id = 'jerarquia_valoraciones_usuario',
        body = { 'jerarquia_valoraciones_usuario': 'valoraciones_usuario','usuarios.fechaControl' : fecha_ejecucion}
)
print("actualizada fecha control de ejecucion:",fecha_ejecucion)

actualizada fecha control de ejecucion: 2021-06-02 00:00:00


In [89]:
mintic_calificaciones['@timestamp'] = now.isoformat()
def doc_generator(df):        
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_id": f"{ 'Calificacion-' + str(document['usuarios.tablero13.siteID']) + '-' + str(document['usuarios.tablero13.fechaCalificacion'])}",
                "_source": filterKeys(document),
            }
salida = helpers.bulk(es, doc_generator(mintic_calificaciones))
print("Fecha: ", now,"- Total calificaciones insertadas en indice principal:",salida[0])

KeyError: 'usuarios.tablero13.siteID'